# EvoWiki: Evaluating LLMs on Evolving Knowledge

 https://github.com/wtangdev/EvoWiki

 Daniel Tytkowski

 Krzysztof Wolny

In [ ]:
import json

data = []
with open('evolved_1hop__with_larger_corpus_retriever_1014.json', 'r', encoding='utf-8') as file:
    for _ in range(15):
        line = file.readline()
        if not line:
            break  
        data.append(json.loads(line))

In [2]:
data[0]

{'values_2019': ['Q75178', 'Q948657', 'Q611644'],
 'values_2401': ['Q75178', 'Q948657', 'Q29182'],
 'values_2405': ['Q75178', 'Q948657', 'Q105070978'],
 'claims_2019': [{'claim_id': 'Q12026446$022B5AA2-665B-4022-8613-4A567DFE6217',
   'qid': 'Q12026446',
   'property_id': 'P39',
   'value': 'Q75178'},
  {'claim_id': 'Q12026446$F642D2BF-1743-402D-A059-8A3D30108C71',
   'qid': 'Q12026446',
   'property_id': 'P39',
   'value': 'Q611644'},
  {'claim_id': 'Q12026446$C7D5271A-34CF-444A-B5B2-0E7E939E2A6F',
   'qid': 'Q12026446',
   'property_id': 'P39',
   'value': 'Q948657'}],
 'claims_2401': [{'claim_id': 'Q12026446$022B5AA2-665B-4022-8613-4A567DFE6217',
   'qid': 'Q12026446',
   'property_id': 'P39',
   'value': 'Q75178'},
  {'claim_id': 'Q12026446$C7D5271A-34CF-444A-B5B2-0E7E939E2A6F',
   'qid': 'Q12026446',
   'property_id': 'P39',
   'value': 'Q948657'},
  {'claim_id': 'Q12026446$C66F2B6D-F44C-43D8-B96D-8F3A5492FFA1',
   'qid': 'Q12026446',
   'property_id': 'P39',
   'value': 'Q29182'}

In [21]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

original_is_autocast = torch.is_autocast_enabled

def fixed_is_autocast(*args, **kwargs):
    return original_is_autocast()

torch.is_autocast_enabled = fixed_is_autocast
print("Compatibility patch applied.")
# deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B
# deepseek-ai/DeepSeek-R1-Distill-Llama-8B
# meta-llama/Llama-3.1-8B-Instruct
model_id = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

print("Downloading/Loading model weights to CPU (this avoids the buffer error)...")

tokenizer = AutoTokenizer.from_pretrained(model_id)

device = torch.device("cpu")

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float32, 
    low_cpu_mem_usage=True
).to(device)

# if torch.backends.mps.is_available():
#     print("Moving model to Apple Silicon GPU (MPS)...")
#     model = model.to("mps")
# elif torch.cuda.is_available():
#     print("Moving model to NVIDIA GPU (CUDA)...")
#     model = model.to("cuda")

print("Model loaded successfully!")

'(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: 834ee4c2-022f-4a60-9ea2-bf2af1522198)')' thrown while requesting HEAD https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Llama-8B/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].


Compatibility patch applied.
Downloading/Loading model weights to CPU (this avoids the buffer error)...


Loading checkpoint shards: 100%|██████████| 2/2 [00:49<00:00, 24.77s/it]


Model loaded successfully!


In [9]:
import gc
def ask_deepseek(question, context=None):
    if context is not None: 
        prompt = f"""Task: Answer the question using the context below with a single word or short phrase.
Context: {context}
Question: {question}
Answer: The correct answer is:
    """
    else: 
        prompt = f"""Task: Answer the question using the context below with a single word or short phrase.
Question: {question}
Answer: The correct answer is:
    """
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs, 
            max_new_tokens=15,
            temperature=0.1,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.encode("\n", add_special_tokens=False)[-1]
        )
    
    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "The correct answer is:" in full_text:
        answer = full_text.split("The correct answer is:")[-1].strip()
    else:
        answer = full_text.strip()

    if "</think>" in answer:
        answer = answer.split("</think>")[-1].strip()
        
    return answer




In [10]:
test_question = data[0]["question"]
test_context = data[0]["golden_context_mentioned"]
test_answer = data[0]["answer"]
test_all_answers = data[0]["all_answer"]


print(f"Question: {test_question}")
print(f"Context: {test_context}")
print(f"Correct answer: {test_answer}")
print(f"All correct answers: {test_all_answers}")

model_answer_context = ask_deepseek(test_question, test_context)
gc.collect()
if torch.backends.mps.is_available():
    torch.mps.empty_cache()
print(f"Model response with given golden context:")
print(model_answer_context)

model_answer = ask_deepseek(test_question)
gc.collect()
if torch.backends.mps.is_available():
    torch.mps.empty_cache()
print(f"Model response without giving context:")
print(model_answer)

Question: What position does Josef Nuzík hold?
Context: ['He was appointed the Metropolitan Archbishop of Olomouc by Pope Francis on 9 February 2024.']
Correct answer: ['Archbishop of Olomouc']
All correct answers: ['auxiliary bishop', 'Archbishop of Olomouc', 'bishop', 'titular bishop', 'Catholic bishop']


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Model response with given golden context:
a) Metropolitan Archbishop
Model response without giving context:
Josef Nuzík is the **[ ]**.
From the context,


In [18]:
import pandas as pd

results_list = []

for i in range(len(data)): 
    json_data_row = data[i]
    test_question = json_data_row["question"]
    test_context = json_data_row["golden_context_mentioned"]
    test_answer = json_data_row["answer"]
    test_all_answers = json_data_row["all_answer"]


    model_answer_context = ask_deepseek(test_question, test_context)
    gc.collect()
    if torch.backends.mps.is_available():
        torch.mps.empty_cache()

    model_answer = ask_deepseek(test_question)
    gc.collect()
    if torch.backends.mps.is_available():
        torch.mps.empty_cache()
    results_list.append({
        "question": test_question,
        "context": test_context,
        "gold_answer": test_answer,
        "all_answers": test_all_answers,
        "model_with_context": model_answer_context,
        "model_no_context": model_answer
    })

df_results = pd.DataFrame(results_list)
df_results.to_csv("evowiki_evaluation_results.csv", index=False)
df_results

,question,context,gold_answer,all_answers,model_with_context,model_no_context
0,What position does Josef Nuzík hold?,[He was appointed the Metropolitan Archbishop ...,[Archbishop of Olomouc],"[auxiliary bishop, Archbishop of Olomouc, bish...",a) Metropolitan Archbishop,"Josef Nuzík is the **[ ]**.\nFrom the context,"
1,What political party is Y. S. Sharmila a membe...,"[Sharmila, is an Indian politician serving as ...",[Indian National Congress],"[YSR Congress Party, Indian National Congress]",A) Indian National Congress,A) Indian National Congress
2,What political party is Metin Kaya a member of?,[The association also started being represente...,[Bündnis Sahra Wagenknecht],"[The Left, Bündnis Sahra Wagenknecht]",A) Alliance 90/The Greens,A) Justice and Development Party (AKP)\n B...
3,What is the occupation of Zeev Glikman?,"[Zeev Glikman (, born 4 August 1958) is an Isr...",[para table tennis player],"[table tennis player, para table tennis player]",A) Farmer,A) Teacher
4,What is the occupation of James N. Gardner?,"[James Nelson Gardner (May 5, 1946 – April 10,...",[lawyer],"[writer, lobbyist, politician, lawyer]",A. Lawyer,A) Teacher
5,What organization is Dominik Perler a member of?,[In 2020 he was elected a member of the Academ...,[Academia Europaea],"[Academia Europaea, Berlin-Brandenburg Academy...",A) European Academy of Sciences,A) European Union
6,What is the occupation of Teet Veispak?,[Veispak worked as a researcher and the direct...,[curator],"[historian, curator]",A) Researcher,A) Teacher
7,What awards has Sandra Hüller been nominated for?,"[For her performance in ""Anatomy of a Fall"" (2...","[César Award for Best Actress, Academy Award f...","[César Award for Best Actress, Academy Award f...",A) Academy Award for Best Actress,A) Grammy
8,What award has Yoav Freund received?,[He was elected an AAAI Fellow in 2008.],[AAAI Fellow],"[Gödel Prize, AAAI Fellow, Paris Kanellakis Aw...",A. AAAI Fellowship,A) Distinguished Service Award
9,What is the occupation of Marthe Kråkstad Joha...,[Marthe Kråkstad Johansen (born 5 January 1999...,[biathlete],"[cross-country skier, biathlete]",A) Teacher,A) Teacher


In [19]:
data = []
with open('stable_1hop_with_retriever_with_mcq.json', 'r', encoding='utf-8') as file:
    for _ in range(15):
        line = file.readline()
        if not line:
            break  
        data.append(json.loads(line))
results_list = []

for i in range(len(data)): 
    json_data_row = data[i]
    test_question = json_data_row["question"]
    test_context = json_data_row["golden_context_mentioned"]
    test_answer = json_data_row["answer"]
    test_all_answers = json_data_row["all_answer"]


    model_answer_context = ask_deepseek(test_question, test_context)
    gc.collect()
    if torch.backends.mps.is_available():
        torch.mps.empty_cache()

    model_answer = ask_deepseek(test_question)
    gc.collect()
    if torch.backends.mps.is_available():
        torch.mps.empty_cache()
    results_list.append({
        "question": test_question,
        "context": test_context,
        "gold_answer": test_answer,
        "all_answers": test_all_answers,
        "model_with_context": model_answer_context,
        "model_no_context": model_answer
    })

df_results_stable = pd.DataFrame(results_list)
df_results_stable.to_csv("evowiki_evaluation_results_stable.csv", index=False)
df_results_stable

,question,context,gold_answer,all_answers,model_with_context,model_no_context
0,What is the country of citizenship for Reda Ha...,[Reda Haikal (born ) is an Egyptian male volle...,[Egypt],[Egypt],A) Egypt,A) United States
1,What are the occupations of Brian Muggleton?,[Mervyn Brian Muggleton (4 September 1941 – 12...,"[cricketer, baseball player]","[cricketer, baseball player]",A) Cricketer,A) Teacher
2,What is the occupation of Karsten Voigt?,[Karsten Dietrich Voigt (born 11 April 1941 in...,[politician],[politician],A) Farmer,A) Farmer
3,What is the country of citizenship of Surendra...,[Surendra Singh Patel is an Indian politician ...,[India],[India],A) India,A) India
4,Who is the employer of Latifa Nabizada?,[Latifa Nabizada is an Afghan helicopter pilot...,[Afghan Air Force],[Afghan Air Force],A) The Afghan Air Force,A) The Ministry of Public Health
5,What event did Brian Lewis participate in?,"[Lewis (born December 5, 1974) is an American ...",[2000 Summer Olympics],[2000 Summer Olympics],A) 4 × 100 m relay,A) A car accident
6,What event did Miroslav Šandor participate in?,[He competed in the men's featherweight event ...,[1980 Summer Olympics],[1980 Summer Olympics],a) Men's featherweight event,A) The 1968 Winter Olympics
7,What is the country of citizenship of Sergei K...,[Sergei Sergeyevich Kramarenko (; born 2 Novem...,[Russia],[Russia],A) Russia,A) United States
8,What is Faisal Iqbal's country of citizenship?,[Faisal Iqbal (born 16 August 1992) is a Pakis...,[Pakistan],[Pakistan],A) Pakistan,A) Pakistan
9,What is the record label of Colleen Green?,[She is currently signed to Hardly Art Records.],[Hardly Art],[Hardly Art],A) Hardly Art Records,A) Green Day


In [20]:

data = []
with open('uncharted_1hop_with_retriever_with_mcq.json', 'r', encoding='utf-8') as file:
    for _ in range(15):
        line = file.readline()
        if not line:
            break  
        data.append(json.loads(line))
results_list = []

for i in range(len(data)): 
    json_data_row = data[i]
    test_question = json_data_row["question"]
    test_context = json_data_row["golden_context_mentioned"]
    test_answer = json_data_row["answer"]
    test_all_answers = json_data_row["all_answer"]


    model_answer_context = ask_deepseek(test_question, test_context)
    gc.collect()
    if torch.backends.mps.is_available():
        torch.mps.empty_cache()

    model_answer = ask_deepseek(test_question)
    gc.collect()
    if torch.backends.mps.is_available():
        torch.mps.empty_cache()
    results_list.append({
        "question": test_question,
        "context": test_context,
        "gold_answer": test_answer,
        "all_answers": test_all_answers,
        "model_with_context": model_answer_context,
        "model_no_context": model_answer
    })

df_results_uncharted = pd.DataFrame(results_list)
df_results_uncharted.to_csv("evowiki_evaluation_results_uncharted.csv", index=False)
df_results_uncharted

,question,context,gold_answer,all_answers,model_with_context,model_no_context
0,What award has Charles Roven received?,"[His film ""American Hustle"" was nominated for ...",[Academy Award for Best Picture],[Academy Award for Best Picture],A) Academy Award for Best Picture,A) Best Director
1,What award has Margaret Fink received?,[Fink was appointed an Officer of the Order of...,[Officer of the Order of Australia],[Officer of the Order of Australia],A) Australia Day Honours,A) National Board of Certified Counselors (NBC...
2,What award has Greg Moriarty received?,[Moriarty was appointed an Officer of the Orde...,[Officer of the Order of Australia],[Officer of the Order of Australia],A) Australia Day Honours,A) Grammy
3,What position does Scott Conger hold?,"[Conger (1928-2011), who served as Jackson's m...",[mayor],[mayor],A. Mayor,A) Mayor
4,What position does Heike Heubach hold?,[She has been a member of the German Bundestag...,[member of the German Bundestag],[member of the German Bundestag],A. Member of the Bundestag,A) Heike Heubach is the CEO of a company.
5,What award has Sharman Stone received?,[Stone was appointed a Member of the Order of ...,[Member of the Order of Australia],[Member of the Order of Australia],A) Australia Day Honours,A) Australian of the Year
6,What award has Judith Kelly received?,[Kelly was appointed an Officer of the Order o...,[Officer of the Order of Australia],[Officer of the Order of Australia],A) Australia Day Honours,A) National Book Award
7,What position does Teófilo Santos hold?,[Teófilo Agostinho Martins Araújo dos Santos (...,[member of the European Parliament],[member of the European Parliament],A) Member of the European Parliament,A) General
8,What position does Ricardo Morgado hold?,[He succeeded Cláudia Aguiar as a member of th...,[member of the European Parliament],[member of the European Parliament],a) Member of the European Parliament,A) General Manager
9,What award has Lara Giddings received?,[Giddings was appointed an Officer of the Orde...,[Officer of the Order of Australia],[Officer of the Order of Australia],A) Australia Day Honours,A) The Nobel Peace Prize
